## Importación de librerías

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sentence_transformers import SentenceTransformer, util
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise
from simpleneighbors import SimpleNeighbors
from tqdm import tqdm
from tqdm import trange
import tensorflow.compat.v2 as tf
import torch
import time
from unidecode import unidecode
import math
import faiss

/home/chronomanteca/PythonEnv/sibea-backend/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-21 13:35:25.215258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 13:35:26.393552: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2023-02-21 13:35:26.393646: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not

## Configuracion notebook

In [6]:
pd.set_option('display.max_columns', None)  

## Cargue de modelos

### MiniLM-L6-v2

In [2]:
#model_minilm = SentenceTransformer('all-MiniLM-L6-v2')
path = './models/model_minilm'
#model_minilm.save(path)
model_minilm = SentenceTransformer(path)

### mpnet-base-v2


In [3]:
#model_mpnet = SentenceTransformer('all-mpnet-base-v2')
path = './models/model_mpnet'
#model_mpnet.save(path)
model_mpnet = SentenceTransformer(path)

### distiluse-base-multilingual-cased-v1

In [4]:
#model_multilingual_distiluse_v1 = SentenceTransformer('distiluse-base-multilingual-cased-v1')
path = './models/model_multilingual_distiluse_v1'
#model_multilingual_distiluse_v1.save(path)
model_multilingual_distiluse_v1 = SentenceTransformer(path)

### distiluse-base-multilingual-cased-v2

In [7]:
#model_multilingual_distiluse_v2 = SentenceTransformer('distiluse-base-multilingual-cased-v2')
path = './models/model_multilingual_distiluse_v2'
#model_multilingual_distiluse_v2.save(path)
model_multilingual_distiluse_v2 = SentenceTransformer(path)

### paraphrase-multilingual-MiniLM-L12-v2

In [6]:
#model_multilingual_minilm = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
path = './models/model_multilingual_minilm'
#model_multilingual_minilm.save(path)
model_multilingual_minilm = SentenceTransformer(path)

### paraphrase-multilingual-mpnet-base-v2

In [7]:
#model_multilingual_mpnet = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
path = './models/model_multilingual_mpnet'
#model_multilingual_mpnet.save(path)
model_multilingual_mpnet = SentenceTransformer(path)


### sentence-encoder-4

In [ ]:
#model_use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
path = './models/universal-sentence-encoder_4'
model_use = hub.KerasLayer(path)
embeddings = model_use(["A long sentence.", "single-word", "http://example.com"])
print(embeddings.shape, embeddings.dtype)


### sentence-enconder-large-5

In [10]:
#model_use_large = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
path = './models/universal-sentence-encoder-large_5'
model_use_large = hub.KerasLayer(path)
embeddings = model_use_large(["A long sentence.", "single-word", "http://example.com"])
print(embeddings.shape, embeddings.dtype)

(3, 512) <dtype: 'float32'>


### sentence-encoder-multilingual-3

In [11]:
#model_use_multilingual = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
path = './models/universal-sentence-encoder-multilingual_3'
model_use_multilingual = hub.KerasLayer(path)
embeddings = model_use_multilingual(["A long sentence.", "single-word", "http://example.com"])
print(embeddings.shape, embeddings.dtype)

(3, 512) <dtype: 'float32'>


### sentence-encoder-multilingual-large-3

In [12]:
#model_use_multilingual_large = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
path = './models/universal-sentence-encoder-multilingual-large_3'
model_use_multilingual_large = hub.KerasLayer(path)
embeddings = model_use_multilingual_large(["A long sentence.", "single-word", "http://example.com"])
print(embeddings.shape, embeddings.dtype)

(3, 512) <dtype: 'float32'>


## Generación e indexado de embeddings

In [13]:
def generate_embeddings(messages):
    result = []
    """
    start_encoding = time.time()
    embeddings_minilm = model_minilm.encode(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_minilm.shape[1])
    index.add(embeddings_minilm)
    faiss.write_index(index, './indexes/index_embeddings_minilm')
    end_indexing = time.time()
    result.append({"model": "minilm", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_minilm})

    start_encoding = time.time()
    embeddings_mpnet = model_mpnet.encode(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_mpnet.shape[1])
    index.add(embeddings_mpnet)
    faiss.write_index(index, './indexes/index_embeddings_mpnet')
    end_indexing = time.time()
    result.append({"model": "mpnet", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_mpnet})

    start_encoding = time.time()
    embeddings_multilingual_distiluse_v1 = model_multilingual_distiluse_v1.encode(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_multilingual_distiluse_v1.shape[1])
    print(index)
    index.add(embeddings_multilingual_distiluse_v1)
    faiss.write_index(index, './indexes/index_embeddings_multilingual_distiluse_v1')
    end_indexing = time.time()
    result.append({"model": "multilingual_distiluse_v1", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_multilingual_distiluse_v1})
    """
    start_encoding = time.time()
    embeddings_multilingual_distiluse_v2 = model_multilingual_distiluse_v2.encode(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_multilingual_distiluse_v2.shape[1])
    index.add(embeddings_multilingual_distiluse_v2)
    faiss.write_index(index, './indexes/index_embeddings_multilingual_distiluse_v2')
    end_indexing = time.time()
    #print(embeddings_multilingual_distiluse_v2.shape())
    result.append({"model": "multilingual_distiluse_v2", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_multilingual_distiluse_v2})
    """
    start_encoding = time.time()
    embeddings_multilingual_minilm = model_multilingual_minilm.encode(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_multilingual_minilm.shape[1])
    index.add(embeddings_multilingual_minilm)
    faiss.write_index(index, './indexes/index_embeddings_multilingual_minilm')
    end_indexing = time.time()
    result.append({"model": "multilingual_minilm", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_multilingual_minilm})

    start_encoding = time.time()
    embeddings_multilingual_mpnet = model_multilingual_mpnet.encode(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_multilingual_mpnet.shape[1])
    index.add(embeddings_multilingual_mpnet)
    faiss.write_index(index, './indexes/index_embeddings_multilingual_mpnet')
    end_indexing = time.time()
    result.append({"model": "multilingual_mpnet", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_multilingual_mpnet})
    
    start_encoding = time.time()
    embeddings_use = model_use(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_use.shape[1])
    index.add(embeddings_use)
    faiss.write_index(index, './indexes/index_embeddings_use')
    end_indexing = time.time()
    result.append({"model": "use", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_use})

    start_encoding = time.time()
    embeddings_use_large = model_use_large(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_use_large.shape[1])
    index.add(embeddings_use_large)
    faiss.write_index(index, './indexes/index_embeddings_use_large')
    end_indexing = time.time()
    result.append({"model": "use_large", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_use_large})

    start_encoding = time.time()
    embeddings_use_multilingual = model_use_multilingual(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_use_multilingual.shape[1])
    index.add(embeddings_use_multilingual)
    faiss.write_index(index, './indexes/index_embeddings_use_multilingual')
    end_indexing = time.time()
    result.append({"model": "use_multilingual", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_use_multilingual})

    start_encoding = time.time()
    embeddings_use_multilingual_large = model_use_multilingual_large(messages)
    end_encoding = time.time()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(embeddings_use_multilingual_large.shape[1])
    index.add(embeddings_use_multilingual_large)
    faiss.write_index(index, './indexes/index_embeddings_use_multilingual_large')
    end_indexing = time.time()
    result.append({"model": "use_multilingual_large", "time_encoding": end_encoding - start_encoding, "time_indexing": end_indexing - start_indexing, "tensor": embeddings_use_multilingual_large})
    """
    
    return result

### Mapa de calor de similaridad

In [11]:
def plot_similarity(labels, features, rotation, ax, title):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd",
      ax=ax
      )
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title(title)

### Mapa de calor todos los modelos

In [12]:
def plot_embeddings_heatmap(embeddings, messages):
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10)) = plt.subplots(ncols=2, nrows=5, sharey=True, figsize=(30, 70), constrained_layout=True)
    for (embedding, ax) in zip(embeddings, [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10]):
        plot_similarity(messages, embedding["tensor"], 90, ax, embedding["model"])

### Mapa de calor tiempos

In [13]:
def plot_embeddings_timing(embeddings, measure="time_encoding"):
    fig, ax = plt.subplots(figsize=(30, 10))
    times = [embedding[measure] for embedding in embeddings]
    ax.bar([embedding["model"] for embedding in embeddings], times)
    if (measure == "time_encoding"):
        ax.set_title("Time to generate embeddings")
    else:
        ax.set_title("Time to index embeddings")
    ax.set_ylabel("Time in seconds")
    ax.set_xlabel("Model")

### Plotting

In [ ]:
plot_embeddings_heatmap(embeddings, messages)
plot_embeddings_timing(embeddings, "time_encoding")
plot_embeddings_timing(embeddings, "time_indexing")

## Prueba

In [18]:
messages = [
    "I like my phone",
    "My cellphone is not good.",
    "I'm going to call someone with my phone",
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",
    "Healthcare system",
    "COVID-19",
    "ADHD",
    "CPAP",
    "Lungs",
    "Psychology",
    "BMI",
    "Fat"
]
embeddings = generate_embeddings(messages)
print(type(embeddings[0].get("tensor")))
tensor = embeddings[0].get("tensor")
print(tensor.shape)



<class 'numpy.ndarray'>
(17, 512)


In [21]:
term = "lluvia"
query = model_multilingual_distiluse_v2.encode(term)
file = open("./queries/test.txt", "w")
file.write(str(query))
file.close()


## Búsqueda

### Generacion de resultados

In [10]:
def search(model, corpus, query, top_k=5, search_method="sbert", model_source="sbert"):
    result = {}
    
    start = time.time()
    if model_source == "sbert":
        query_embeddings = model.encode([query])
        corpus_embeddings = corpus["tensor"]
    elif model_source == "tf":
        query_embeddings = model([query]).numpy()
        corpus_embeddings = corpus["tensor"].numpy()
    
    
    if search_method == "sbert":
        hits = util.semantic_search(query_embeddings, corpus_embeddings, top_k=top_k)
        top_results = np.array([hit['corpus_id'] for hit in hits[0]])
    elif search_method == "torch":
        cos_scores = util.cos_sim(query_embeddings, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k).indices.tolist()
    elif search_method == "numpy":
        cos_scores = util.cos_sim(query_embeddings, corpus_embeddings)[0]
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    elif search_method == "faiss":
        index = faiss.read_index("./indexes/index_embeddings_" + corpus["model"])
        top_results = index.search(query_embeddings, top_k)[1][0]
    elif search_method == "tf":
        cos_scores = tf.keras.losses.cosine_similarity(query_embeddings, corpus_embeddings, axis=1)
        top_results = tf.math.top_k(cos_scores, k=top_k)
        top_results = (top_results.indices.numpy(), top_results.values.numpy())[0]
    

    end = time.time()

    result = {
        "results": top_results,
        "time": end - start,
        "model": corpus["model"],
        "search_method": search_method,
    }
    return result

### Busqueda con todos los métodos de búsqueda

In [11]:
def search_all(query, k, embeddings):
    total = []
    
    results = []
    results.append(search(model_multilingual_distiluse_v1, embeddings[2], query, k, "sbert", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v1, embeddings[2], query, k, "torch", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v1, embeddings[2], query, k, "numpy", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v1, embeddings[2], query, k, "faiss", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v1, embeddings[2], query, k, "tf", model_source="sbert"))
    total.append(results)

    results = []
    results.append(search(model_multilingual_distiluse_v2, embeddings[3], query, k, "sbert", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v2, embeddings[3], query, k, "torch", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v2, embeddings[3], query, k, "numpy", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v2, embeddings[3], query, k, "faiss", model_source="sbert"))
    results.append(search(model_multilingual_distiluse_v2, embeddings[3], query, k, "tf", model_source="sbert"))
    total.append(results)
    
    results = []
    results.append(search(model_multilingual_minilm, embeddings[4], query, k, "sbert", model_source="sbert"))
    results.append(search(model_multilingual_minilm, embeddings[4], query, k, "torch", model_source="sbert"))
    results.append(search(model_multilingual_minilm, embeddings[4], query, k, "numpy", model_source="sbert"))
    results.append(search(model_multilingual_minilm, embeddings[4], query, k, "faiss", model_source="sbert"))
    results.append(search(model_multilingual_minilm, embeddings[4], query, k, "tf", model_source="sbert"))
    total.append(results)
    """
    results = []
    results.append(search(model_multilingual_mpnet, embeddings[5], query, k, "sbert", model_source="sbert"))
    results.append(search(model_multilingual_mpnet, embeddings[5], query, k, "torch", model_source="sbert"))
    results.append(search(model_multilingual_mpnet, embeddings[5], query, k, "numpy", model_source="sbert"))
    results.append(search(model_multilingual_mpnet, embeddings[5], query, k, "faiss", model_source="sbert"))
    results.append(search(model_multilingual_mpnet, embeddings[5], query, k, "tf", model_source="sbert"))
    total.append(results)
   
    results = []
    results.append(search(model_use_multilingual, embeddings[8], query, k, "sbert", model_source="tf"))
    results.append(search(model_use_multilingual, embeddings[8], query, k, "torch", model_source="tf"))
    results.append(search(model_use_multilingual, embeddings[8], query, k, "numpy", model_source="tf"))
    results.append(search(model_use_multilingual, embeddings[8], query, k, "faiss", model_source="tf"))
    results.append(search(model_use_multilingual, embeddings[8], query, k, "tf", model_source="tf"))
    total.append(results)

    results = []
    results.append(search(model_use_multilingual_large, embeddings[9], query, k, "sbert", model_source="tf"))
    results.append(search(model_use_multilingual_large, embeddings[9], query, k, "torch", model_source="tf"))
    results.append(search(model_use_multilingual_large, embeddings[9], query, k, "numpy", model_source="tf"))
    results.append(search(model_use_multilingual_large, embeddings[9], query, k, "faiss", model_source="tf"))
    results.append(search(model_use_multilingual_large, embeddings[9], query, k, "tf", model_source="tf"))
    total.append(results)
    """

    return total

### Mapa de calor de tiempos

In [1]:
def plot_search_timing(results):
    times = [[result["time"] for result in results] for results in results]
    plot = sns.heatmap(times, annot=True, fmt=".3f", xticklabels=["sbert", "torch", "numpy", "faiss", "tf"], yticklabels=["multilingual_distiluse_v1", "multilingual_distiluse_v2", "multilingual_minilm", "multilingual_mpnet", "use_multilingual", "use_multilingual_large"])
    plot.set_title("Time to search")
    plot.set_ylabel("Model")
    plot.set_xlabel("Search method")

## Resultados de búsqueda

In [12]:
def search_results(results, messages, model_index):
    for index, result in enumerate(results):
        print("Model #" + str(index) + " - " + result[0]['model'])
        for i, indexes in enumerate(result):
            if (index == model_index):
                print(indexes["search_method"] + " - ", end="")
                print(str([messages[i] for i in indexes["results"]]) + "\n")

### Prueba resultados

In [ ]:
results = search_all("Pobreza", 5, embeddings)
#plot_search_timing(results)
search_results(results, messages, 2)

## Titulos investigaciones

### Importacion y limpieza de datos

In [13]:
inv_df = pd.read_excel('./xlsx/InvestigarPUJ.xlsx', sheet_name='Hoja1', converters={'ID PROYECTO':str})
siap_df = pd.read_excel('./xlsx/Descriptores SIAP 2023.xlsx', sheet_name='SIAP ', converters={'ID Proy':str})
inv_full_df = pd.read_excel('./xlsx/Descriptores SIAP 2023.xlsx', sheet_name='InvestigarPUJ', converters={'Id':str})

inv_full_df.drop("Año", axis=1, inplace=True)

datasets = [inv_df, siap_df, inv_full_df]
for dataset in datasets:
    dataset.replace('\\N', np.NaN, inplace=True)
    dataset.replace('null', np.NaN, inplace=True)
    dataset.replace('nan', np.NaN, inplace=True)
    dataset.replace('N/A', np.NaN, inplace=True)


merged = inv_df.merge(siap_df, left_on="ID PROYECTO", right_on="ID Proy", how="left")
df = merged.merge(inv_full_df, left_on="ID PROYECTO", right_on="Id", how="left")

df = df.groupby("ID PROYECTO").agg(lambda x: list(set(x))).applymap(lambda x: ', '.join(str(y) for y in x if str(y) != 'nan') if isinstance(x, list) else x)
df = df.reset_index()
df.columns = [unidecode(x.lower().strip().replace(" ", "_").replace("__", "_").replace(".", "")) for x in df.columns]

df.replace('', np.NaN, inplace=True)

df["descripcion"] = df["descripcion_x"].fillna("") + df["descripcion_y"].fillna("")

df.drop("titulo_x", axis=1, inplace=True)
df.drop("titulo_y", axis=1, inplace=True)
df.drop("id_proy", axis=1, inplace=True)
df.drop("id", axis=1, inplace=True)
df.drop("financiador", axis=1, inplace=True)
df.drop("tipo_propuesta", axis=1, inplace=True)
df.drop("descripcion_x", axis=1, inplace=True)
df.drop("descripcion_y", axis=1, inplace=True)
df.drop("id_propt", axis=1, inplace=True)
df.drop("f_inic_real", axis=1, inplace=True)
df.drop("f_final_real", axis=1, inplace=True)
df.drop("facultad", axis=1, inplace=True)
df.drop("departamento", axis=1, inplace=True)
df.drop("estado_proyecto", axis=1, inplace=True)
df.drop("nombre", axis=1, inplace=True)
df.drop("cantidad", axis=1, inplace=True)

print(df.columns)
print(df.shape)

#df[df["id_proyecto"] == "000000000007161"]
#df[df["id_proyecto"] == "004438"]
#df[df["id_proyecto"] == "20104"]

df["corpus"] = df["titulo_del_proyecto"].fillna("") + " "  + \
df["nombre_facultad"].str.split().str[-1].fillna("") + " " + \
df["nombre_del_departamento"].str.split().str[-1].fillna("") + " " + \
df["descripcion"].fillna("") + " " + \
df["resumen"].fillna("") + " " + \
df["objetivos"].fillna("") + " " + \
df["metodologia"].fillna("") + " " + \
df["gran_area"].fillna("") + " " + \
df["objetivo_socioeconomico"].fillna("") + " " + \
df["palabras_clave"].fillna("")

df["corpus"] = df["corpus"].replace(r'\n',' ', regex=True).str.strip()

messages = df["corpus"].head(1000).to_numpy()

Index(['id_proyecto', 'id_propuesta', 'tipo_de_propuesta',
       'fecha_de_negociacion', 'ano', 'titulo_del_proyecto',
       'fecha_inicial_real', 'fecha_final_real', 'nombre_facultad',
       'nombre_del_departamento', 'convocatoria', 'nombre_patrocinador',
       'nombre_grupo_de_investigacion', 'miembro_del_equipo',
       'tipo_de_documento', 'numero_de_documento', 'nombres_y_apellidos',
       'rol_en_el_proyecto', 'porcentaje_dedicacion', 'codigo_presupuesto',
       'codigo_contrato_secre_juridica', 'valor_aprobado_patrocinador',
       'contrapartida_terceros', 'contrapartida_rec_propi',
       'contrapartida_rec_nuevo', 'rec_nuevos_unidade_academica',
       'valor_contrapartida_total', 'valor_total', 'estado', 'codigo_tipo',
       'convocatoria_nombre_corto', 'convocatoria', 'tipo_de_actividad',
       'fecha_estimada_inicio', 'fecha_estimada_fin',
       'id_empleado_responsable_puj', 'nombre_responsable_puj',
       'departamento_responsable_puj', 'id_departamento_respon

In [31]:
df["corpus"].to_csv("./corpus.csv", index=False)

In [32]:
df[df["id_proyecto"] == "000000000007161"]

,id_proyecto,id_propuesta,tipo_de_propuesta,fecha_de_negociacion,ano,titulo_del_proyecto,fecha_inicial_real,fecha_final_real,nombre_facultad,nombre_del_departamento,convocatoria,nombre_patrocinador,nombre_grupo_de_investigacion,miembro_del_equipo,tipo_de_documento,numero_de_documento,nombres_y_apellidos,rol_en_el_proyecto,porcentaje_dedicacion,codigo_presupuesto,codigo_contrato_secre_juridica,valor_aprobado_patrocinador,contrapartida_terceros,contrapartida_rec_propi,contrapartida_rec_nuevo,rec_nuevos_unidade_academica,valor_contrapartida_total,valor_total,estado,codigo_tipo,convocatoria_nombre_corto,convocatoria,tipo_de_actividad,fecha_estimada_inicio,fecha_estimada_fin,id_empleado_responsable_puj,nombre_responsable_puj,departamento_responsable_puj,id_departamento_responsable_puj,facultad_responsable_puj,nombre_financiador_principal,resumen,objetivos,metodologia,categoria,producto,valor_solicitado_a_financiador_principal,valor_total_financiadores,tipo_de_investigacion,gran_area,objetivo_socioeconomico,palabras_clave,aprobado?,descripcion,corpus
710,000000000007161,00007029,PUE,2015-07-27 00:00:00,2015,Constitución del Centro de Excelencia y Apropi...,2015-11-06 00:00:00,2019-05-31 00:00:00,Facultad de Ingeniería,Dpto. de Electrónica,NaN,"DEPARTAMENTO ADMINISTRATIVO DE CIENCIA,","CEPIT: Sistemas de Control, Electrónica de Pot...","00010153314, 00010123188, 0031139, 00010032943...","SE, CC","80095329, 1010164261, 16865144, 16771142, 1015...","Mendez Chaves,Diego, Cotrino Badillo,Carlos, C...","COIN, PI","0.0, 10.0, 100.0, 50.0",121300C0101103,Contrato fp44842-502-2015,6354717309.0,2164600000.0,10400000.0,70000000.0,200000000,0,8799717309.0,Pendiente Compromiso,"OBJGE, OBJES",NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Diseñar, montar e iniciar la operación de un C...",Constitución del Centro de Excelencia y Apropi...


### Descriptores

In [33]:
categories = df["nombre_facultad"].groupby(df["nombre_facultad"]).count().sort_values(ascending=False)
top20 = df["titulo_del_proyecto"].drop_duplicates().str.split(expand=True).stack().value_counts()[:20]

### Generacion embeddings

In [20]:
embeddings = generate_embeddings(messages)

### Guardando tensores generados

In [10]:
minilm_tensor = embeddings[0].get('tensor')
mpnet_tensor = embeddings[1].get('tensor')
multilingual_distiluse_v1_tensor = embeddings[2].get('tensor')
multilingual_distiluse_v2_tensor = embeddings[3].get('tensor')
multilingual_minilm_tensor = embeddings[4].get('tensor')

np.save('./tensors/binary/minilm_tensor.npy', minilm_tensor)
np.save('./tensors/binary/mpnet_tensor.npy', mpnet_tensor)
np.save('./tensors/binary/multilingual_distiluse_v1_tensor.npy', multilingual_distiluse_v1_tensor)
np.save('./tensors/binary/multilingual_distiluse_v2_tensor.npy', multilingual_distiluse_v2_tensor)
np.save('./tensors/binary/multilingual_minilm_tensor.npy', multilingual_minilm_tensor)

### Cargando tensores generados

In [ ]:
embeddings = []
embeddings.append({"model":"minilm","tensor":np.load('./tensor/binary/minilm_tensor.npy')})
embeddings.append({"model":"mpnet","tensor":np.load('./tensor/binary/mpnet_tensor.npy')})
embeddings.append({"model":"multilingual_distiluse_v1","tensor":np.load('./tensor/binary/multilingual_distiluse_v1_tensor.npy')})
embeddings.append({"model":"multilingual_distiluse_v2","tensor":np.load('./tensor/binary/multilingual_distiluse_v2_tensor.npy')})
embeddings.append({"model":"multilingual_minilm","tensor":np.load('./tensor/binary/multilingual_minilm_tensor.npy')})
print(embeddings)

### Guardando como CSV

In [18]:
np.savetxt('./tensors/csv/minilm_tensor.csv', minilm_tensor,delimiter=",")
np.savetxt('./tensors/csv/mpnet_tensor.csv', mpnet_tensor,delimiter=",")
np.savetxt('./tensors/csv/multilingual_distiluse_v1_tensor.csv', multilingual_distiluse_v1_tensor,delimiter=",")
np.savetxt('./tensors/csv/multilingual_distiluse_v2_tensor.csv', multilingual_distiluse_v2_tensor,delimiter=",")
np.savetxt('./tensors/csv/multilingual_minilm_tensor.csv', multilingual_minilm_tensor,delimiter=",")

In [42]:
arr = embeddings[0].get("tensor")
print(arr.shape)
arr = embeddings[1].get("tensor")
print(arr.shape)
arr = embeddings[2].get("tensor")
print(arr.shape)
arr = embeddings[3].get("tensor")
print(arr.shape)
arr = embeddings[4].get("tensor")
print(arr.shape)
arr = embeddings[5].get("tensor")
print(arr.shape)

(1000, 384)
(1000, 768)
(1000, 512)
(1000, 512)
(1000, 384)


IndexError: list index out of range

In [ ]:
plot_embeddings_timing(embeddings, "time_encoding")
plot_embeddings_timing(embeddings, "time_indexing")

### Intento de guardar los embeddings en disco


In [18]:
embeddings = np.loadtxt('embeddings.csv', delimiter=',')
"""
print(type(embeddings[0]))
print(type(embeddings[0].get('tensor')))
f = open('embeddings.csv', 'a')
for embedding in embeddings:
    for key in embedding:
        #print(embedding.get('tensor'))
        f.write("\n")
        np.savetxt(f,embedding.get('tensor'),delimiter=',')
"""






ValueError: the number of columns changed from 384 to 768 at row 4001; use `usecols` to select a subset and avoid this error

### Prueba busqueda

In [ ]:
results = search_all("Explotacion infantil", 10, embeddings)
plot_search_timing(results)
search_results(results, messages, 2)